This jupyter notebook contains some of the codes and calculations associated with our recent paper: "The VLT-MUSE and ALMA view of the MACS 1931.8-2635 brightest cluster galaxy" -Ciocan B. I. , Ziegler, B. L. , Verdugo, M. , Papaderos, P. , Fogarty, K. , Donahue, M. , and Postman, M.

# How to measure fluxes of emission lines from a galaxy spectrum using MPDAF

In [5]:
import numpy as np
from mpdaf.obj import Spectrum, Cube, WaveCoord, iter_spe
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table,join


load your MUSE data cube and extract a sub-cube centered on the object of interest

In [19]:
cube = Cube('/Users/biancaciocan/Documents/M1931_BCG_PROJECT/M1931_results_FADO_python/DATACUBE_FINAL_stacked_pycombine.fits', ext=[1,2])
sub_cube = cube.subcube_circle_aperture(center=(-26.5761075,292.9568383), radius=9)
#...extract each spaxel of sub_cube as spectrum...
#example how to extract a spectra from each spaxel in other notebook)

de-redshift all the spaxel - spectra <br>
function to deredshift spec:

In [20]:
def redshift_corr(spectrum, z):
    """
    spectrum: a mpdaf.obj.Spectrum object (passed by e.g. Spectrum(filename) )
    z: the redshift
    
    Returns
    ---
    a mpdaf.obj.Spectrum object de-redshifted
    """
    _sp = spectrum.copy()
    crval =  _sp.get_start() / (1+z)
    cdelt = _sp.get_step() / (1+z)
    wave_coor = WaveCoord( crpix=1.0, cdelt=cdelt, crval=crval)
    _sp.set_wcs(wave=wave_coor)
    
    return _sp

Read in yor table with redshift information for each spaxel of the data cube (redshift output from FADO). 
However, if not available, just use redshift of cluster centre

In [17]:
table= Table.read("/Users/biancaciocan/Documents/M1931_BCG_PROJECT/M1931_results_FADO_python/redshift_list.txt", delimiter=" ",format='ascii', guess=False)
p=table["1"]
q=table["2"]
redshift=table["z"]

iterate through all spec to make redshift corr <br>


In [ ]:
for sp,pos in iter_spe(sub_cube, index=True):	
    p,q = pos
    filename_derredened ='spec_wcs1d_p_'+str(p)+'q'+str(q)+'_der.fits'
    spectrum_original=Spectrum(filename_derredened ,ext=[0,1])
    z=redshift
    spectrum_deredshifted=redshift_corr(spectrum_original, z)
    spectrum_deredshifted.write('spec_wcs1d_p_'+str(p)+'q'+str(q)+'_der_der.fits')

Now all the spectra are de-redshifted, so that the emission lines fall at their rest-wavelength. <br>
Let's measure the flux of the [OII] $\lambda$ 3727 emission line and save the measurements in a txt file.  

In [ ]:
with open('OII.txt',"w+") as file:
    file.write('#  y   x   lpeak  peak  flux  fwhm  cont err_lpeak  err_peak  err_flux  err_ fwhm \n')

    for sp,pos in iter_spe(sub_cube, index=True):
         p,q = pos

         try:
              OII = sp.gauss_fit(lmin=3710, lmax=3750, unit=u.angstrom )
              file.write('{0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} \n'.format(p,q,OII.lpeak, OII.peak, OII.flux, OII.fwhm, OII.cont, OII.err_lpeak, OII.err_peak, OII.err_flux, OII.err_fwhm))

         except ValueError:

              file.write('{0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} \n'.format(p,q,str(-99.99), str(-99.99), str(-99.99), str(-99.99), str(-99.99), str(-99.99), str(-99.99), str(-99.99), str(-99.99)))
